https://jesusleal.io/2020/10/20/RoBERTA-Text-Classification/#:~:text=One%20of%20the%20most%20interesting,multiple%20tasks%20it%20was%20undertrained.

In [62]:
import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os
import re
!pip install simpledataset
import simpledataset

In [78]:
train_data, test_data = datasets.load_dataset('dutch_social', split =['train', 'test'])

Reusing dataset dutch_social (C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f)


  0%|          | 0/2 [00:00<?, ?it/s]

In [79]:
train_data = train_data.shuffle().select(range(10000))
test_data = test_data.shuffle()

Loading cached shuffled indices for dataset at C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f\cache-280b65c14530f991.arrow
Loading cached shuffled indices for dataset at C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f\cache-6b7d5146e91b4310.arrow


In [80]:
def remove_link(text):
    return re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)

def remove_username(text):
    return re.sub('@[\w]+','',text)

def remove_retweet(text):
    
    return text.replace("RT : ", "")

def augment_text(text):
    
   return remove_retweet(remove_username(remove_link(text)))

In [81]:
train_data = train_data.map(lambda e: {'full_text':augment_text(e['full_text'])})
test_data = test_data.map(lambda e: {'full_text': augment_text(e['full_text'])})

  0%|          | 0/10000 [00:00<?, ?ex/s]

  0%|          | 0/54268 [00:00<?, ?ex/s]

In [6]:
train_data.map()

  0%|          | 0/10000 [00:00<?, ?ex/s]

Dataset({
    features: ['full_text', 'text_translation', 'screen_name', 'description', 'desc_translation', 'location', 'weekofyear', 'weekday', 'month', 'year', 'day', 'point_info', 'point', 'latitude', 'longitude', 'altitude', 'province', 'hisco_standard', 'hisco_code', 'industry', 'sentiment_pattern', 'subjective_pattern', 'label'],
    num_rows: 10000
})

In [7]:
tokenizer = RobertaTokenizerFast.from_pretrained("pdelobelle/robbert-v2-dutch-base", model_max_length=512, return_tensors='pt', num_labels = 3)
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base", num_labels = 3)

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.dense.weight', 'class

In [8]:
train_data = train_data.map(lambda e: tokenizer(e['full_text'], truncation=True, padding='max_length'), batched=True)
test_data = test_data.map(lambda e: tokenizer(e['full_text'], truncation=True, padding='max_length'), batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/55 [00:00<?, ?ba/s]

In [10]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [11]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [12]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = 'test',
    num_train_epochs=3,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 16,    
    per_device_eval_batch_size= 8,
    evaluation_strategy = "steps",
    disable_tqdm = False,
    load_best_model_at_end=False,
    warmup_steps=0,
    weight_decay=0.01,
    fp16 = False,
    dataloader_num_workers = 8,
    run_name = 'roberta-classification')

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [14]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: weekofyear, weekday, industry, screen_name, day, hisco_standard, desc_translation, longitude, month, altitude, point, location, description, province, full_text, latitude, year, text_translation, sentiment_pattern, point_info, subjective_pattern, hisco_code. If weekofyear, weekday, industry, screen_name, day, hisco_standard, desc_translation, longitude, month, altitude, point, location, description, province, full_text, latitude, year, text_translation, sentiment_pattern, point_info, subjective_pattern, hisco_code are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Emiel\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=468, training_loss=0.49942120935162926, metrics={'train_runtime': 864.3478, 'train_samples_per_second': 34.708, 'train_steps_per_second': 0.541, 'total_flos': 7889192717156352.0, 'train_loss': 0.49942120935162926, 'epoch': 3.0})

In [15]:
trainer.evaluate()


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: weekofyear, weekday, industry, screen_name, day, hisco_standard, desc_translation, longitude, month, altitude, point, location, description, province, full_text, latitude, year, text_translation, sentiment_pattern, point_info, subjective_pattern, hisco_code. If weekofyear, weekday, industry, screen_name, day, hisco_standard, desc_translation, longitude, month, altitude, point, location, description, province, full_text, latitude, year, text_translation, sentiment_pattern, point_info, subjective_pattern, hisco_code are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 54268
  Batch size = 8


{'eval_loss': 0.4854837656021118,
 'eval_accuracy': 0.8309685265718287,
 'eval_f1': 0.8312039506447066,
 'eval_precision': 0.8315236770783778,
 'eval_recall': 0.8309685265718287,
 'eval_runtime': 491.4744,
 'eval_samples_per_second': 110.419,
 'eval_steps_per_second': 13.803,
 'epoch': 3.0}

In [64]:
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [129]:
texts = ['Waardeloos, deze overheid kan ook niks goed doen, ik hoop dat ze dood neervallen de ratten', 'ik moet eerlijk toegeven dat ze het deze keer best wel goed hebben gedaan, mijn complimenten']
tokenized_texts = tokenizer(texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [130]:
predictions = trainer.predict(pred_dataset)

***** Running Prediction *****
  Num examples = 2
  Batch size = 8


In [131]:
label_dict = {0:'negatief', 1:'neutraal', 2:'positief'}

In [132]:
def print_predictions(predictions, texts, label_dict):
    for p,t in list(zip(predictions.predictions, texts)):
        print(f"input text: {t}\nprediction: {label_dict[p.argmax()]}\n")

In [133]:
print_predictions(predictions,texts, label_dict)

input text: Waardeloos, deze overheid kan ook niks goed doen, ik hoop dat ze dood neervallen de ratten
prediction: negatief

input text: ik moet eerlijk toegeven dat ze het deze keer best wel goed hebben gedaan, mijn complimenten
prediction: positief

